In [1]:
# Obtiene la climatología media para 1970-2000 a partir de
# datos del CRU y de WorldClim.

import os


import xarray as xr

In [2]:
origen = ["CRU", "WC"]
loc = ["datos", "resultados"]

# Año de inicio y de fin de climatología, inclusive.
yr_i = "1970"
yr_f = "2000"

# Nombre de variables en cada conjunto de archivos.
var = [("tmx", "tmn"), ("tmin", "tmax")]

# Decide si quitar o no quitar "stn", solo quita para CRU.
drop = [True, False]

In [4]:
# Itera para cada conjunto de datos de origen (CRU y WC).
for i in range(1, len(origen)):
    ds = []
    
    path_d = "../../" + loc[i] + "/" + origen[0] + "/"
    path_r = "../../" + loc[1] + "/" + origen[0] + "/"

    # Se enlistan las subcarpetas del directorio.
    fname = os.listdir(path_d)
    fname.reverse

    # Si no existe la carpeta, la crea.
    if not os.path.exists(path_r):
        os.mkdir(path_r)

    # Se obtiene el promedio mensual para cada conjunto de datos.
    for j, value in enumerate(fname[0:2]): 
        # Se cargan y concatenan todos los archivos correspondientes
        # para cada subcarpeta de WC_nc.
        ds.append(xr.load_dataset(path_d + fname[j]))
        print(ds[0])

        # Decide si quitar o no quitar "stn", solo quita para CRU.
        if drop[i]: ds[j] = ds[j].drop("stn")

        # Se selecciona el periodo deseado.
        ds[j] = ds[j].sel(time = slice(yr_i, yr_f))

        # Se hace que las variables sean compatibles.
        ds[j] = ds[j].rename_vars({var[i][j]: "T"})
        # Se agrega una dimensión para poder concatenar los archivos.
        ds[j] = ds[j].expand_dims(num =  [i])

    # Se concatenan los archivos.
    ds_conc = xr.combine_nested(ds, concat_dim = "num")
    # Se promedia la temperatura maxima y mínima.
    ds_conc = ds_conc.mean("num")

    # Se obtiene la media mensual.
    ds_conc = ds_conc.groupby("time.month").mean()

    # Se guarda el netCDF.
    ds_conc.to_netcdf(path_r + origen[i] + "_clim_" + str(yr_i)
        + "_" + str(yr_f) + "_monthly.nc")


IndexError: list index out of range